In [ ]:
# Google Colaboratory를 사용할 때는 다음 주석을 해제하고 실행하기 바랍니다.

In [ ]:
# # 다음을 실행하면 authorization code 입력을 요청받습니다.
# # 출력된 링크를 클릭하고 Google 계정으로 로그인한 뒤,
# # authorization code를 복사해서 붙여 넣습니다.
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# working_dir = 'MLSys_100Knocks' #　※※ 여러분이 만든 폴더 경로가 다를 때는 다음을 변경합니다. ※※
# path = f'/content/drive/MyDrive/{working_dir}/MainChapter/chapter06'
# os.chdir(path)

In [ ]:
# Jupyter notebook ipywidgets 활성화
# for jupyter notebook (virtualenv 사용 시)
#!jupyter nbextension enable --user --py widgetsnbextension

# for jupyter lab
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager

# 6장 머신러닝용 데이터 가공을 위한 테크닉 10

### 테크닉 51: 데이터 가공을 위한 밑준비를 하자

In [ ]:
import os

data_dir = 'data'
input_dir = os.path.join(data_dir, '0_input')
output_dir = os.path.join(data_dir, '1_output')
master_dir = os.path.join(data_dir, '99_master')
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)

#####  **반드시 데이터를 해당 폴더에 배치합니다**

#### Mac 또는 Google Colaborator 등, 환경에 따라서는 파일을 얻는 순서가 연월순이 되지 않을 수 있습니다.
#### 그럴 때는 `tbl_order_paths = sorted(tbl_order_paths)`를 추가하기 바랍니다.

In [ ]:
import glob

tbl_order_file = os.path.join(input_dir, 'tbl_order_*.csv')
tbl_order_paths = glob.glob(tbl_order_file)
tbl_order_paths

### 테크닉 52: 데이터를 로딩하고 데이터 가공 방향성을 검토하자

In [ ]:
import pandas as pd

m_area_file = 'm_area.csv'
m_store_file = 'm_store.csv'
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
m_area.head(3)

In [ ]:
tbl_order_path = tbl_order_paths[0]
print(f'데이터 로딩: {tbl_order_path}')
order_data = pd.read_csv(tbl_order_path)
print(f'데이터 건수: {len(order_data)}')
order_data.head(3)

### 테크닉 53: 1개월분 데이터로 기본적인 가공을 하자

In [ ]:
order_data = order_data.loc[order_data['store_id'] != 999]

order_data = pd.merge(order_data, m_store, on='store_id', how='left')
order_data = pd.merge(order_data, m_area, on='area_cd', how='left')

order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'delivery'
order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'takeout'

order_data.loc[order_data['status'] == 0, 'status_name'] = '주문 접수'
order_data.loc[order_data['status'] == 1, 'status_name'] = '계산 완료'
order_data.loc[order_data['status'] == 2, 'status_name'] = '배달 완료'
order_data.loc[order_data['status'] == 9, 'status_name'] = '주문 취소'
order_data.head(3)

In [ ]:
order_data.isna().sum()

### 테크닉 54: 머신러닝용 변수를 만들자

In [ ]:
def calc_delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds() / 60


order_data.loc[:, 'order_accept_datetime'] = \
    pd.to_datetime(order_data['order_accept_date'])
order_data.loc[:, 'delivered_datetime'] = \
    pd.to_datetime(order_data['delivered_date'])
order_data.loc[:, 'delta'] = order_data[[
    'order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis=1)
order_data.head(3)

In [ ]:
order_data.loc[:, 'order_accept_hour'] = \
    order_data['order_accept_datetime'].dt.hour
order_data.loc[:, 'order_accept_weekday'] = \
    order_data['order_accept_datetime'].dt.weekday
order_data.loc[order_data['order_accept_weekday'] >= 5,
               'weekday_info'] = '휴일'
order_data.loc[order_data['order_accept_weekday'] < 5,
               'weekday_info'] = '평일'
order_data.head(3)

### 테크닉 55: 매장 단위로 집계해서 변수를 만들자

In [ ]:
store_data = order_data.groupby(['store_name']).count()[['order_id']]
store_f = order_data.loc[
    (order_data['status_name'] == "배달 완료") |
    (order_data['status_name'] == "결제 완료")
    ].groupby(['store_name']).count()[['order_id']]
store_c = order_data.loc[
    order_data['status_name'] == "주문 취소"
    ].groupby(['store_name']).count()[['order_id']]
store_d = order_data.loc[
    order_data['takeout_name'] == "delivery"
    ].groupby(['store_name']).count()[['order_id']]
store_t = order_data.loc[
    order_data['takeout_name'] == "takeout"
    ].groupby(['store_name']).count()[['order_id']]

store_weekday = order_data.loc[
    order_data['weekday_info'] == "평일"
    ].groupby(['store_name']).count()[['order_id']]
store_weekend = order_data.loc[
    order_data['weekday_info'] == "휴일"
    ].groupby(['store_name']).count()[['order_id']]

In [ ]:
times = order_data['order_accept_hour'].unique()
store_time = []
for time in times:
    time_tmp = order_data.loc[
        order_data['order_accept_hour'] == time
        ].groupby(['store_name']).count()[['order_id']]
    time_tmp.columns = [f'order_time_{time}']
    store_time.append(time_tmp)
store_time = pd.concat(store_time, axis=1)
store_time.head(3)

In [ ]:
store_delta = order_data.loc[
    order_data['status_name'] != "주문 취소"
    ].groupby(['store_name']).mean()[['delta']]
store_data.columns = ['order']
store_f.columns = ['order_fin']
store_c.columns = ['order_cancel']
store_d.columns = ['order_delivery']
store_t.columns = ['order_takeout']
store_weekday.columns = ['order_weekday']
store_weekend.columns = ['order_weekend']
store_delta.columns = ['delta_avg']
store_data = pd.concat([
    store_data, store_f, store_c, store_d, store_t,
    store_weekday, store_weekend, store_time, store_delta], axis=1)
store_data.head(3)

### 테크닉 56: 데이터 가공과 매장별 집계를 함수로 실행하자

In [ ]:
def data_processing(order_data):
    order_data = order_data.loc[order_data['store_id'] != 999]
    order_data = pd.merge(order_data, m_store, on='store_id', how='left')
    order_data = pd.merge(order_data, m_area, on='area_cd', how='left')
    order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'delivery'
    order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'takeout'
    order_data.loc[order_data['status'] == 0, 'status_name'] = '주문 접수'
    order_data.loc[order_data['status'] == 1, 'status_name'] = '결제 완료'
    order_data.loc[order_data['status'] == 2, 'status_name'] = '배달 완료'
    order_data.loc[order_data['status'] == 9, 'status_name'] = '주문 취소'

    order_data.loc[:, 'order_accept_datetime'] = pd.to_datetime(
        order_data['order_accept_date'])
    order_data.loc[:, 'delivered_datetime'] = pd.to_datetime(
        order_data['delivered_date'])
    order_data.loc[:, 'delta'] = order_data[
        ['order_accept_datetime', 'delivered_datetime']
    ].apply(calc_delta, axis=1)
    order_data.loc[:, 'order_accept_hour'] = order_data[
        'order_accept_datetime'].dt.hour
    order_data.loc[:, 'order_accept_weekday'] = order_data[
        'order_accept_datetime'].dt.weekday
    order_data.loc[order_data['order_accept_weekday'] >= 5,
                   'weekday_info'] = '휴일'
    order_data.loc[order_data['order_accept_weekday'] < 5,
                   'weekday_info'] = '평일'

    store_data = order_data.groupby(['store_name']).count()[['order_id']]
    store_f = order_data.loc[
        (order_data['status_name'] == "배달 완료") |
        (order_data['status_name'] == "결제 완료")
        ].groupby(['store_name']).count()[['order_id']]
    store_c = order_data.loc[
        order_data['status_name'] == "주문 취소"
        ].groupby(['store_name']).count()[['order_id']]
    store_d = order_data.loc[
        order_data['takeout_name'] == "delivery"
        ].groupby(['store_name']).count()[['order_id']]
    store_t = order_data.loc[
        order_data['takeout_name'] == "takeout"
        ].groupby(['store_name']).count()[['order_id']]
    store_weekday = order_data.loc[
        order_data['weekday_info'] == "평일"
        ].groupby(['store_name']).count()[['order_id']]
    store_weekend = order_data.loc[
        order_data['weekday_info'] == "휴일"
        ].groupby(['store_name']).count()[['order_id']]
    times = order_data['order_accept_hour'].unique()
    store_time = []

    for time in times:
        time_tmp = order_data.loc[
            order_data['order_accept_hour'] == time
            ].groupby(['store_name']).count()[['order_id']]
        time_tmp.columns = [f'order_time_{time}']
        store_time.append(time_tmp)

    store_time = pd.concat(store_time, axis=1)
    store_delta = order_data.loc[
        order_data['status_name'] != "주문 취소"
        ].groupby(['store_name']).mean()[['delta']]
    store_data.columns = ['order']
    store_f.columns = ['order_fin']
    store_c.columns = ['order_cancel']
    store_d.columns = ['order_delivery']
    store_t.columns = ['order_takeout']
    store_delta.columns = ['delta_avg']
    store_weekday.columns = ['order_weekday']
    store_weekend.columns = ['order_weekend']
    store_data = pd.concat(
        [store_data, store_f, store_c, store_d, store_t,
         store_weekday, store_weekend, store_time, store_delta], axis=1)
    return store_data

In [ ]:
tbl_order_path = tbl_order_paths[0]
print(f'데이터 로딩: {tbl_order_path}')
order_data = pd.read_csv(tbl_order_path)
store_data = data_processing(order_data)
store_data.head(3)

### 테크닉 57: 모든 데이터를 로딩하고 데이터를 가공하자

In [ ]:
store_all = []
for tbl_order_path in tbl_order_paths:
    print(f'데이터 로딩: {tbl_order_path}')
    tg_ym = tbl_order_path.split('_')[-1][:6]
    order_data = pd.read_csv(tbl_order_path)
    store_data = data_processing(order_data)
    store_data.loc[:,'year_month'] = tg_ym
    store_data.reset_index(drop=False, inplace=True)
    store_all.append(store_data)

store_all = pd.concat(store_all, ignore_index=True)
display(store_all.head(3))
display(store_all.tail(3))
store_monthly_name = 'store_monthly_data.csv'
store_all.to_csv(os.path.join(output_dir, store_monthly_name), index=False)

### 테크닉 58: 목적 변수를 만들자

In [ ]:
y = store_all[
    ['store_name', 'year_month', 'order_weekday', 'order_weekend']].copy()
y.loc[:, 'one_month_ago'] = pd.to_datetime(y['year_month'], format='%Y%m')

from dateutil.relativedelta import relativedelta

y.loc[:, 'one_month_ago'] = y['one_month_ago'].map(
    lambda x: x - relativedelta(months=1))
y.loc[:, 'one_month_ago'] = y['one_month_ago'].dt.strftime('%Y%m')
y.head(3)

In [ ]:
y_one_month_ago = y.copy()
y_one_month_ago.rename(columns={
    'order_weekday': 'order_weekday_one_month_ago',
    'order_weekend': 'order_weekend_one_month_ago',
    'year_month': 'year_month_for_join'}, inplace=True)
y = pd.merge(y, y_one_month_ago[['store_name', 'year_month_for_join',
                                 'order_weekday_one_month_ago',
                                 'order_weekend_one_month_ago']],
             left_on=['store_name', 'one_month_ago'],
             right_on=['store_name', 'year_month_for_join'], how='left')
y.loc[y['store_name'] == '가덕해안로점']

In [ ]:
y.dropna(inplace=True)
y.loc[y['order_weekday'] - y['order_weekday_one_month_ago'] > 0, 'y_weekday'] = 1
y.loc[y['order_weekday'] - y['order_weekday_one_month_ago'] <= 0, 'y_weekday'] = 0
y.loc[y['order_weekend'] - y['order_weekend_one_month_ago'] > 0, 'y_weekend'] = 1
y.loc[y['order_weekend'] - y['order_weekend_one_month_ago'] <= 0, 'y_weekend'] = 0
y.head(3)

### 테크닉 59: 설명 변수와 목적 변수를 연결해 머신러닝용 데이터를 완성하자

In [ ]:
y.rename(columns={'year_month': 'target_year_month'},
         inplace=True)
y = y[['store_name', 'target_year_month', 'one_month_ago',
       'y_weekday', 'y_weekend']].copy()
ml_data = pd.merge(y, store_all,
                   left_on=['store_name', 'one_month_ago'],
                   right_on=['store_name', 'year_month'],
                   how='left')
ml_data.head()

In [ ]:
del ml_data["target_year_month"]
del ml_data["one_month_ago"]
ml_data.head()

### 테크닉 60: 머신러닝용 데이터를 확인하고 출력하자

In [ ]:
ml_data.isna().sum()

In [ ]:
display(ml_data.groupby('y_weekday').count()[['store_name']])
display(ml_data.groupby('y_weekend').count()[['store_name']])

In [ ]:
ml_data.to_csv(os.path.join(output_dir, 'ml_base_data.csv'), index=False)